In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from scipy.spatial.distance import cdist

In [ ]:
# from tf.keras.models import Sequential  # This does not work!
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, GRU, Embedding
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

In [ ]:
import imdb #Dataset

In [ ]:
imdb.maybe_download_and_extract()

In [ ]:
x_train_text, y_train = imdb.load_data(train=True)
x_test_text, y_test = imdb.load_data(train=False)

In [ ]:
data_text = x_train_text + x_test_text

In [ ]:
num_words = 10000

In [ ]:
tokenizer = Tokenizer(num_words=num_words)

In [ ]:
%%time
tokenizer.fit_on_texts(data_text)

In [ ]:
if num_words is None:
    num_words = len(tokenizer.word_index)

In [ ]:
tokenizer.word_index

In [ ]:
x_train_tokens = tokenizer.texts_to_sequences(x_train_text)

In [ ]:
np.array(x_train_tokens[1])

In [ ]:
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]
num_tokens = np.array(num_tokens)

In [ ]:
np.mean(num_tokens)

In [ ]:
np.max(num_tokens)

In [ ]:
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens)
max_tokens

In [ ]:
np.sum(num_tokens < max_tokens) / len(num_tokens)

In [ ]:
pad = 'pre'


In [ ]:
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens,
                            padding=pad, truncating=pad)

In [ ]:
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens,
                           padding=pad, truncating=pad)

In [ ]:
x_train_pad.shape

In [ ]:
x_test_pad.shape

In [ ]:
np.array(x_train_tokens[1])

In [ ]:
x_train_pad[1]

In [ ]:
idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))

In [ ]:
def tokens_to_string(tokens):
    # Map from tokens back to words.
    words = [inverse_map[token] for token in tokens if token != 0]
    
    # Concatenate all words.
    text = " ".join(words)

    return text

In [ ]:
x_train_text[1]

In [ ]:
tokens_to_string(x_train_tokens[1])

In [ ]:
model = Sequential()

In [ ]:
embedding_size = 8

In [ ]:
model.add(Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_tokens,
                    name='layer_embedding'))

In [ ]:
model.add(GRU(units=16, return_sequences=True))

In [ ]:
model.add(GRU(units=8, return_sequences=True))

In [ ]:
model.add(GRU(units=4))

In [ ]:
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
%%time
model.fit(x_train_pad, y_train,
          validation_split=0.05, epochs=3, batch_size=64)

In [ ]:
%%time
result = model.evaluate(x_test_pad, y_test)

In [ ]:
print("Accuracy: {0:.2%}".format(result[1]))